In [6]:
import sys
from pathlib import Path
import pandas as pd
import geopandas as gpd
from fuzzywuzzy import process

# Add project root to Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import project paths
from src.helpers.config import DATA_DIR
from src.helpers.utils import read_csv

# Convert wojewodztwa do GeoJSON

In [ ]:
# Define the path to the shapefile
shp_path = Path(DATA_DIR / "geo_data" / "A01_Granice_wojewodztw.shp")

# Read the shapefile into a GeoDataFrame
gdf = gpd.read_file(shp_path)

# Check the structure of the data
print("Preview of the shapefile data:")
print(gdf.head())

# Define the output path for the GeoJSON file
geojson_path = Path(DATA_DIR / "wojewodztwa.geojson")

# Save the data to GeoJSON format
gdf.to_file(geojson_path, driver="GeoJSON")

print(f"File successfully saved as {geojson_path}")


In [10]:
# Load the shapefile containing the boundaries of Polish regions
map = gpd.read_file(Path(DATA_DIR / "wojewodztwa.geojson"))
name_mapping = {
    "maÅ\x82opolskie": "małopolskie",
    "Å\x82Ã³dzkie": "łódzkie",
    "dolnoÅ\x9blÄ\x85skie": "dolnośląskie",
    "Å\x9bwiÄ\x99tokrzyskie": "świętokrzyskie",
    "warmiÅ\x84sko-mazurskie": "warmińsko-mazurskie",
    "Å\x9blÄ\x85skie": "śląskie",
}

# Replace incorrect names with correct ones
map["JPT_NAZWA_"] = map["JPT_NAZWA_"].replace(name_mapping)
map.to_file(Path(DATA_DIR / "wojewodztwa.geojson"), driver="GeoJSON", encoding="utf-8")


# Convert powiaty do GeoJSON

In [63]:
# Define the path to the shapefile
shp_path = Path(DATA_DIR / "geo_data" / "A02_Granice_powiatow.shp")

# Read the shapefile into a GeoDataFrame
gdf = gpd.read_file(shp_path)

# Define the output path for the GeoJSON file
geojson_path = Path(DATA_DIR / "powiaty.geojson")

# Save the data to GeoJSON format
gdf.to_file(geojson_path, driver="GeoJSON")

with open(
    Path(DATA_DIR / "powiaty.geojson"), "r", encoding="utf-8", errors="replace"
) as f:
    content = f.read()

# Convert to correct encoding
content = content.encode("latin1").decode("utf-8")

# Save to as the same file
with open(Path(DATA_DIR / "powiaty.geojson"), "w", encoding="utf-8") as f:
    f.write(content)


In [64]:
# Load województwa GeoJSON
wojewodztwa = gpd.read_file(Path(DATA_DIR / "wojewodztwa.geojson"))
powiaty = gpd.read_file(Path(DATA_DIR / "powiaty.geojson"))

# Ensure both datasets have the same CRS
powiaty = powiaty.to_crs(wojewodztwa.crs)

# Remove the 'index_right' column if it exists in either dataframe
if "index_right" in powiaty.columns:
    powiaty.drop(columns=["index_right"], inplace=True)

if "index_right" in wojewodztwa.columns:
    wojewodztwa.drop(columns=["index_right"], inplace=True)

# Perform spatial join with suffixes to differentiate columns
powiaty = gpd.sjoin(
    powiaty,
    wojewodztwa[["JPT_NAZWA_", "geometry"]],
    how="left",
    predicate="within",
    lsuffix="POW",
    rsuffix="WOJ",
)
powiaty.columns = powiaty.columns.str.replace("__", "_", regex=True)

# Standardize names by converting to lowercase and removing leading spaces
powiaty["JPT_NAZWA_POW"] = (
    powiaty["JPT_NAZWA_POW"].str.lower().str.replace("powiat ", "").str.strip()
)
powiaty["JPT_NAZWA_WOJ"] = powiaty["JPT_NAZWA_WOJ"].str.lower().str.strip()

# Save the cleaned data to GeoJSON format
powiaty.to_file(Path(DATA_DIR / "powiaty.geojson"), driver="GeoJSON", encoding="utf-8")


In [23]:
# Load the shapefile containing the boundaries of Polish regions (powiaty)
powiaty = gpd.read_file(Path(DATA_DIR / "powiaty.geojson"))

# Load the processed data (example CSV file containing powiat and województwo names)
df = pd.read_csv(Path(DATA_DIR / "processed" / "swiad_pow_poradnia_okulistyczna.csv"))

df["Powiat"] = df["Powiat"].str.lower().str.strip()
df["Województwo"] = df["Województwo"].str.lower().str.strip()

# Create a combined name for matching
powiaty["full_name"] = powiaty["JPT_NAZWA_POW"] + powiaty["JPT_NAZWA_WOJ"].fillna(
    ""
).apply(lambda x: f"_{x}" if x else "")
df["full_name"] = df["Powiat"] + df["Województwo"].fillna("").apply(
    lambda x: f"_{x}" if x else ""
)

# Ensure all values in full_name columns are strings and handle NaN values
powiaty["full_name"] = powiaty["full_name"].fillna("").astype(str)
df["full_name"] = df["full_name"].fillna("").astype(str)

# Compare unique names after standardization
geo_json_unique = set(powiaty["full_name"].unique())
clinic_unique = set(df["full_name"].unique())

# Find unmatched names after standardization
unmatched_clinic = clinic_unique - geo_json_unique
unmatched_geo_json = geo_json_unique - clinic_unique

print("Unmatched in clinic data:", unmatched_clinic)
print("Unmatched in GeoJSON:", unmatched_geo_json)


# Function to find best matches using fuzzy matching and return unmatched lists
def create_mapping(geojson_list, clinic_list):
    mapping = {}
    unmatched_geojson = []
    unmatched_clinic = list(clinic_list)  # Create a mutable copy of clinic list

    for geojson_name in geojson_list:
        best_match, score = process.extractOne(geojson_name, clinic_list)
        if (
            score > 90
        ):  # Accept matches with a higher score threshold for better accuracy
            mapping[geojson_name] = best_match
            unmatched_clinic.remove(best_match)  # Remove matched clinic names
        else:
            mapping[geojson_name] = None
            unmatched_geojson.append(geojson_name)

    # Filter out None values from mapping to get unmatched GeoJSON names
    unmatched_geojson += [k for k, v in mapping.items() if v is None]

    return mapping, unmatched_geojson, unmatched_clinic


# Create mapping between unmatched datasets
powiat_mapping, unmatched_geojson, unmatched_clinic = create_mapping(
    unmatched_geo_json, unmatched_clinic
)

print("\nUnmatched GeoJSON names:")
print(unmatched_geojson)

print("\nUnmatched Clinic names:")
print(unmatched_clinic)

Unmatched in clinic data: {'m. koszalin_zachodniopomorskie', 'pucki_pomorskie', 'm. gdynia_pomorskie', 'm. rzeszów_podkarpackie', 'm. katowice_śląskie', 'm. opole_opolskie', 'm. tarnów_małopolskie', 'm. lublin_lubelskie', 'm. chorzów_śląskie', 'm. bielsko-biała_śląskie', 'nowotarski_małopolskie', 'm. st. warszawa_mazowieckie', 'm. wrocław_dolnośląskie', 'm. sosnowiec_śląskie', 'm. świnoujście_zachodniopomorskie', 'kłodzki_dolnośląskie', 'm. ostrołęka_mazowieckie', 'żywiecki_śląskie', 'm. tarnobrzeg_podkarpackie', 'm. jastrzębie-zdrój_śląskie', 'm. zamość_lubelskie', 'm. łomża_podlaskie', 'm. leszno_wielkopolskie', 'm. tychy_śląskie', 'm. biała podlaska_lubelskie', 'tatrzański_małopolskie', 'm. kielce_świętokrzyskie', 'm. poznań_wielkopolskie', 'm. częstochowa_śląskie', 'm. bydgoszcz_kujawsko-pomorskie', 'm. toruń_kujawsko-pomorskie', 'm. słupsk_pomorskie', 'm. jelenia góra_dolnośląskie', 'm. skierniewice_łódzkie', 'm. sopot_pomorskie', 'm. jaworzno_śląskie', 'm. radom_mazowieckie', 'ża

In [24]:
# Manual correction for unmatched powiat
powiat_mapping.update(
    {
        "piotrkowski_łódzkie": "m. piotrków trybunalski_łódzkie",
        "karkonoski_dolnośląskie": "jeleniogórski_dolnośląskie",
        "kaliski_wielkopolskie": "m. kalisz_wielkopolskie",
        "siedlecki_mazowieckie": "m. siedlce_mazowieckie",
        "chełmski_lubelskie": "m. chełm_lubelskie",
        "leski_podkarpackie": "m. lesko_podkarpackie",  # Lack data in clinic
        "skierniewicki_łódzkie": "m. skierniewice_łódzkie",
        "koniński_wielkopolskie": "m. konin_wielkopolskie",
        "przemyski_podkarpackie": "m. przemyśl_podkarpackie",
        "łomżyński_podlaskie": "m. łomża_podlaskie",
        "koszaliński_zachodniopomorskie": "m. koszalin_zachodniopomorskie",
        "leszczyński_wielkopolskie": "m. leszno_wielkopolskie",
        "ostrołęcki_mazowieckie": "m. ostrołęka_mazowieckie",
        "suwalski_podlaskie": "m. suwałki_podlaskie",
        "włocławski_kujawsko-pomorskie": "m. włocławek_kujawsko-pomorskie",
        "żagański_lubuskie": "żagański_lubuskie",
        "prudnicki_opolskie": "prudnicki_opolskie",
        "głubczycki_opolskie": "głubczycki_opolskie",
        "rawicki_wielkopolskie": "rawicki_wielkopolskie",
        "gryficki_zachodniopomorskie": "gryficki_zachodniopomorskie",
        "tomaszowski_lubelskie": "tomaszowski_lubelskie",
        "milicki_dolnośląskie": "milicki_dolnośląskie",
        "kłodzki": "kłodzki_dolnośląskie",
        "milicki": "milicki_dolnośląskie",
        "zgorzelecki": "zgorzelecki_dolnośląskie",
        "tomaszowski": "tomaszowski_lubelskie",
        "żagański": "żagański_lubuskie",
        "mławski": "mławski_mazowieckie",
        "nowotarski": "nowotarski_małopolskie",
        "tatrzański": "tatrzański_małopolskie",
        "brzeski": "brzeski_opolskie",
        "głubczycki": "głubczycki_opolskie",
        "prudnicki": "prudnicki_opolskie",
        "słupski": "m. słupsk_pomorskie",
        "pucki": "pucki_pomorskie",
        "rawicki": "rawicki_wielkopolskie",
        "gryficki": "gryficki_zachodniopomorskie",
        "świnoujście": "m. świnoujście_zachodniopomorskie",
        "wodzisławski": "wodzisławski_śląskie",
        "żywiecki": "żywiecki_śląskie",
    }
)

# Apply mapping to the GeoDataFrame
powiaty["full_name"] = (
    powiaty["full_name"].map(powiat_mapping).fillna(powiaty["full_name"])
)

# Save the updated mapping to a new file
powiaty.to_file(Path(DATA_DIR / "powiaty_mapped.geojson"), driver="GeoJSON")

In [25]:
# Identify entries in df that are not in powiaty
clinic_missing_entries = df[~df["full_name"].isin(powiaty["full_name"])]
clinic_missing_entries["full_name"].unique()

array([], dtype=object)

In [26]:
# Identify entries in df that are not in powiaty
powiaty_missing_entries = powiaty[~powiaty["full_name"].isin(df["full_name"])]
powiaty_missing_entries["full_name"].unique()

array(['m. lesko_podkarpackie'], dtype=object)

In [33]:
df["Powiat"].unique().shape

(356,)

In [35]:
powiaty["full_name"].unique().shape

(367,)

In [36]:
print(powiaty[powiaty["full_name"] == "bartoszycki_warmińsko-mazurskie"]["geometry"])


95    POLYGON ((20.30409 54.3474, 20.30413 54.34744,...
Name: geometry, dtype: geometry
